In [1]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import time
import xgboost as xgb

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [2]:
# Load data
train_filled = pl.read_parquet('data/train.parquet').drop('__index_level_0__')
from scripts.feature_enigeer import clean_fill_and_cast_columns
train_filled = clean_fill_and_cast_columns(train_filled)

✅ 共找到 73 個字串欄位
✅ 共找到 48 個數值欄位
✅ 共找到 4 個布林欄位
✅ 已完成空字串處理、缺失補值、布林轉0/1


In [2]:
# Load data
test_filled = pl.read_parquet('data/test.parquet').drop('__index_level_0__').with_columns(pl.lit(0, dtype=pl.Int64).alias("selected"))
from scripts.feature_enigeer import clean_fill_and_cast_columns


test_filled = clean_fill_and_cast_columns(test_filled, test= True)

✅ 共找到 57 個字串欄位
✅ 共找到 48 個數值欄位
✅ 共找到 4 個布林欄位
✅ test=True: 已將 10 個duration欄位轉str並填'missing'
✅ 已完成空字串處理、缺失補值、布林轉0/1


# price rank

In [ ]:
from scripts.feature_enigeer import build_price_features
# 執行價格特徵工程，並存檔
price_features = build_price_features(
    train_filled,
    output_dir="data/extra_features/train/"
)

# 查看結果
print(price_features)


In [ ]:
from scripts.feature_enigeer import build_price_features
# 執行價格特徵工程，並存檔
price_features = build_price_features(
    test_filled,
    output_dir="data/extra_features/test/"
)

# 查看結果
print(price_features)


# 處理duration

In [ ]:
import polars as pl
from scripts.feature_enigeer import build_duration_features
# 所需欄位
duration_cols = [
    "legs0_duration",
    "legs1_duration",
    "legs0_segments0_duration",
    "legs0_segments1_duration",
    "legs0_segments2_duration",
    "legs0_segments3_duration",
    "legs1_segments0_duration",
    "legs1_segments1_duration",
    "legs1_segments2_duration",
    "legs1_segments3_duration"
]

needed_cols = ["Id", "ranker_id", "totalPrice"] + duration_cols

# 篩選
train_filled = train_filled.select([c for c in needed_cols if c in train_filled.columns])

# 再交給 build_duration_features
duration_features = build_duration_features(
    train_filled,
    output_dir="data/extra_features/train/"
)


In [ ]:
import polars as pl
from scripts.feature_enigeer import build_duration_features

# 再交給 build_duration_features
duration_features = build_duration_features(
    test_filled,
    output_dir="data/extra_features/test/"
)


# frequent_flyer_features code

In [3]:
import polars as pl
from scripts.feature_enigeer import build_frequent_flyer_match_features
# 所需欄位
needed_cols = [
        "Id", "ranker_id", "frequentFlyer",
        "legs0_duration", "legs1_duration",
        *[f"{s}_{t}" for s in [
            "legs0_segments0", "legs0_segments1", "legs0_segments2", "legs0_segments3",
            "legs1_segments0", "legs1_segments1", "legs1_segments2", "legs1_segments3"
        ] for t in ["marketingCarrier_code", "operatingCarrier_code", "duration"]]
    ]


# 篩選
train_filled = train_filled.select([c for c in needed_cols if c in train_filled.columns])

# 再交給 build_duration_features
ff_features = build_frequent_flyer_match_features(
    train_filled,
    output_dir="data/extra_features/train/"
)


✅ 已儲存 Parquet: data/extra_features/train/3_frequent_flyer_features.parquet
✅ 已完成 frequentFlyer 特徵 + match 特徵 + duration 特徵生成


In [9]:
import polars as pl
from scripts.feature_enigeer import build_frequent_flyer_match_features
# 所需欄位
needed_cols = [
        "Id", "ranker_id", "frequentFlyer",
        "legs0_duration", "legs1_duration",
        *[f"{s}_{t}" for s in [
            "legs0_segments0", "legs0_segments1", "legs0_segments2", "legs0_segments3",
            "legs1_segments0", "legs1_segments1", "legs1_segments2", "legs1_segments3"
        ] for t in ["marketingCarrier_code", "operatingCarrier_code", "duration"]]
    ]


# 篩選
test_filled = test_filled.select([c for c in needed_cols if c in test_filled.columns])

# 再交給 build_duration_features
ff_features = build_frequent_flyer_match_features(
    test_filled,
    output_dir="data/extra_features/test/"
)


✅ 已儲存 Parquet: data/extra_features/test/3_frequent_flyer_features.parquet
✅ 已完成 frequentFlyer 特徵 + match 特徵 + duration 特徵生成


# Bagges

In [ ]:
from scripts.feature_enigeer import build_baggage_fee_features

baggage_fee_df = build_baggage_fee_features(
    df=train_filled.select([
        "Id",
        "totalPrice",
        'ranker_id',
        "legs0_segments0_baggageAllowance_quantity",
        "legs1_segments0_baggageAllowance_quantity",
        "miniRules0_monetaryAmount",
        "miniRules1_monetaryAmount"
    ]),
    output_dir="data/extra_features/train/"
)


In [3]:
from scripts.feature_enigeer import build_baggage_fee_features

baggage_fee_df = build_baggage_fee_features(
    df=test_filled.select([
        "Id",
        "totalPrice",
        'ranker_id',
        "legs0_segments0_baggageAllowance_quantity",
        "legs1_segments0_baggageAllowance_quantity",
        "miniRules0_monetaryAmount",
        "miniRules1_monetaryAmount"
    ]),
    output_dir="data/extra_features/test/"
)


✅ 已儲存 Parquet: data/extra_features/test/4_baggage_fee_features.parquet


# cabinClass

In [ ]:
from scripts.feature_enigeer import build_cabin_features
train_filled = train_filled.select([
        "Id",
        *[f"legs{i}_segments{j}_cabinClass" for i in [0,1] for j in range(4)],
        *[f"legs{i}_segments{j}_duration" for i in [0,1] for j in range(4)]
    ])

cabin_features_df = build_cabin_features(
    df=train_filled,
    output_dir="data/extra_features/train/"
)


In [4]:
from scripts.feature_enigeer import build_cabin_features
test_filled = test_filled.select([
        "Id",
        *[f"legs{i}_segments{j}_cabinClass" for i in [0,1] for j in range(4)],
        *[f"legs{i}_segments{j}_duration" for i in [0,1] for j in range(4)]
    ])

cabin_features_df = build_cabin_features(
    df=test_filled,
    output_dir="data/extra_features/test/"
)


sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
/Users/deweywang/Desktop/GitHub/FlightRank/scripts/feature_enigeer.py:592: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  pl.struct(all_duration_cols + all_cabin_cols)

✅ 已儲存 Parquet: data/extra_features/test/5_cabin_features.parquet


# add time

In [ ]:
from scripts.feature_enigeer import build_time_features
train_filled = train_filled.select(["Id", "requestDate", "legs0_departureAt", "legs0_arrivalAt", "legs1_departureAt", "legs1_arrivalAt"])

df_time_features = build_time_features(
    df=train_filled,
    output_dir="data/extra_features/train/"
)


In [3]:
from scripts.feature_enigeer import build_time_features
test_filled = test_filled.select(["Id", "requestDate", "legs0_departureAt", "legs0_arrivalAt", "legs1_departureAt", "legs1_arrivalAt"])

df_time_features = build_time_features(
    df=test_filled,
    output_dir="data/extra_features/test/"
)


✅ 已儲存 Parquet: data/extra_features/test/6_time_features.parquet
✅ 所有時間特徵已生成完成


# build_corporate_access_route_features

In [ ]:
from scripts.feature_enigeer import build_corporate_access_route_features
df_features = build_corporate_access_route_features(
    df=train_filled.select(["Id", "corporateTariffCode", "pricingInfo_isAccessTP", "searchRoute"]),
    output_dir="data/extra_features/train/"
)


In [3]:
from scripts.feature_enigeer import build_corporate_access_route_features
df_features = build_corporate_access_route_features(
    df=test_filled.select(["Id", "corporateTariffCode", "pricingInfo_isAccessTP", "searchRoute"]),
    output_dir="data/extra_features/test/"
)


✅ 已儲存 Parquet: data/extra_features/test/7_corporate_access_route_features.parquet
✅ 已完成 corporate/access/route 特徵生成


# 加上是否直飛跟轉機幾次

In [ ]:
from scripts.feature_enigeer import build_transfer_features
transfer_features_df = build_transfer_features(
    df=train_filled.select([
        "Id", "ranker_id", "legs0_duration", "legs1_duration",
        "legs0_segments1_departureFrom_airport_iata",
        "legs0_segments2_departureFrom_airport_iata",
        "legs0_segments3_departureFrom_airport_iata",
        "legs1_segments1_departureFrom_airport_iata",
        "legs1_segments2_departureFrom_airport_iata",
        "legs1_segments3_departureFrom_airport_iata"
    ]),
    output_dir="data/extra_features/train/"
)


In [4]:
from scripts.feature_enigeer import build_transfer_features
transfer_features_df = build_transfer_features(
    df=test_filled.select([
        "Id", "ranker_id", "legs0_duration", "legs1_duration",
        "legs0_segments1_departureFrom_airport_iata",
        "legs0_segments2_departureFrom_airport_iata",
        "legs0_segments3_departureFrom_airport_iata",
        "legs1_segments1_departureFrom_airport_iata",
        "legs1_segments2_departureFrom_airport_iata",
        "legs1_segments3_departureFrom_airport_iata"
    ]),
    output_dir="data/extra_features/test/"
)


✅ 已儲存 Parquet: data/extra_features/test/8_transfer_features.parquet
✅ 已完成轉機特徵生成


# Carrier

In [3]:
from scripts.feature_enigeer import build_carrier_consistency_features
required_columns = [
    # 主鍵
    "Id",
    "ranker_id",
    # legs0 轉機判斷
    "legs0_segments1_departureFrom_airport_iata",
    "legs0_segments2_departureFrom_airport_iata",
    "legs0_segments3_departureFrom_airport_iata",
    # legs1 轉機判斷
    "legs1_segments1_departureFrom_airport_iata",
    "legs1_segments2_departureFrom_airport_iata",
    "legs1_segments3_departureFrom_airport_iata",
    # legs0 Marketing
    "legs0_segments0_marketingCarrier_code",
    "legs0_segments1_marketingCarrier_code",
    "legs0_segments2_marketingCarrier_code",
    "legs0_segments3_marketingCarrier_code",
    # legs1 Marketing
    "legs1_segments0_marketingCarrier_code",
    "legs1_segments1_marketingCarrier_code",
    "legs1_segments2_marketingCarrier_code",
    "legs1_segments3_marketingCarrier_code"
]

train_filled = train_filled.select(required_columns)
import pickle
with open("data/extra_features/transform_config_rank.pkl", "rb") as f:
    config = pickle.load(f)

carrier_df = build_carrier_consistency_features(
    df=train_filled,
    output_dir="data/extra_features/train/",
    transform_config=config
)


✅ 正在共用carrier encoding處理 ['legs0_main_carrier', 'legs1_main_carrier']
✅ 已儲存 Parquet: data/extra_features/train/9_carrier_consistency_features.parquet
✅ 已完成主Carrier一致性與轉機次數特徵


In [ ]:
from scripts.feature_enigeer import build_carrier_consistency_features
required_columns = [
    # 主鍵
    "Id",
    "ranker_id",
    # legs0 轉機判斷
    "legs0_segments1_departureFrom_airport_iata",
    "legs0_segments2_departureFrom_airport_iata",
    "legs0_segments3_departureFrom_airport_iata",
    # legs1 轉機判斷
    "legs1_segments1_departureFrom_airport_iata",
    "legs1_segments2_departureFrom_airport_iata",
    "legs1_segments3_departureFrom_airport_iata",
    # legs0 Marketing
    "legs0_segments0_marketingCarrier_code",
    "legs0_segments1_marketingCarrier_code",
    "legs0_segments2_marketingCarrier_code",
    "legs0_segments3_marketingCarrier_code",
    # legs1 Marketing
    "legs1_segments0_marketingCarrier_code",
    "legs1_segments1_marketingCarrier_code",
    "legs1_segments2_marketingCarrier_code",
    "legs1_segments3_marketingCarrier_code"
]

test_filled = test_filled.select(required_columns)
import pickle
with open("data/extra_features/transform_config_rank.pkl", "rb") as f:
    config = pickle.load(f)

carrier_df = build_carrier_consistency_features(
    df=test_filled,
    output_dir="data/extra_features/test/",
    transform_config=config
)


✅ 正在共用carrier encoding處理 ['legs0_main_carrier', 'legs1_main_carrier']
✅ 已儲存 Parquet: data/extra_features/test/9_carrier_consistency_features.parquet
✅ 已完成主Carrier一致性與轉機次數特徵


# encoding category features.

In [4]:
from scripts.feature_enigeer import build_label_encoding_features
# ✅ 先把所有 columns 列出
all_cols = train_filled.columns

# ✅ Collect relevant columns
selected_cols = []
# ✅ Collect relevant columns + Id
selected_cols = ["Id"]  # 一定要先放Id

# Aircraft code
selected_cols += [c for c in all_cols if c.endswith("_aircraft_code")]

# Flight Number
selected_cols += [c for c in all_cols if c.endswith("_flightNumber")]

# Airport / City
selected_cols += [c for c in all_cols if "_arrivalTo_airport_" in c or "_departureFrom_airport_" in c]

# Carrier
selected_cols += [c for c in all_cols if c.endswith("_marketingCarrier_code") or c.endswith("_operatingCarrier_code")]

# searchRoute
selected_cols.append("searchRoute")

# frequentFlyer
selected_cols.append("frequentFlyer")

# 去除重複
selected_cols = list(dict.fromkeys(selected_cols))

print("✅ build_label_encoding_features 需要的欄位：", selected_cols)

train_filled = train_filled.select(selected_cols)
import pickle
with open("data/extra_features/transform_config_rank.pkl", "rb") as f:
    config = pickle.load(f)

# 執行編碼
df_encoded, config = build_label_encoding_features(
    df=train_filled,
    output_dir="data/extra_features/train/",
    transform_config=config
)


✅ build_label_encoding_features 需要的欄位： ['Id', 'legs0_segments0_aircraft_code', 'legs0_segments1_aircraft_code', 'legs0_segments2_aircraft_code', 'legs0_segments3_aircraft_code', 'legs1_segments0_aircraft_code', 'legs1_segments1_aircraft_code', 'legs1_segments2_aircraft_code', 'legs1_segments3_aircraft_code', 'legs0_segments0_flightNumber', 'legs0_segments1_flightNumber', 'legs0_segments2_flightNumber', 'legs0_segments3_flightNumber', 'legs1_segments0_flightNumber', 'legs1_segments1_flightNumber', 'legs1_segments2_flightNumber', 'legs1_segments3_flightNumber', 'legs0_segments0_arrivalTo_airport_city_iata', 'legs0_segments0_arrivalTo_airport_iata', 'legs0_segments0_departureFrom_airport_iata', 'legs0_segments1_arrivalTo_airport_city_iata', 'legs0_segments1_arrivalTo_airport_iata', 'legs0_segments1_departureFrom_airport_iata', 'legs0_segments2_arrivalTo_airport_city_iata', 'legs0_segments2_arrivalTo_airport_iata', 'legs0_segments2_departureFrom_airport_iata', 'legs0_segments3_arrivalTo_ai

In [3]:
from scripts.feature_enigeer import build_label_encoding_features
import pickle

# ✅ 先把所有 columns 列出
all_cols = test_filled.columns

# ✅ Collect relevant columns
selected_cols = []
# ✅ Collect relevant columns + Id
selected_cols = ["Id"]  # 一定要先放Id

# Aircraft code
selected_cols += [c for c in all_cols if c.endswith("_aircraft_code")]

# Flight Number
selected_cols += [c for c in all_cols if c.endswith("_flightNumber")]

# Airport / City
selected_cols += [c for c in all_cols if "_arrivalTo_airport_" in c or "_departureFrom_airport_" in c]

# Carrier
selected_cols += [c for c in all_cols if c.endswith("_marketingCarrier_code") or c.endswith("_operatingCarrier_code")]

# searchRoute
selected_cols.append("searchRoute")

# frequentFlyer
selected_cols.append("frequentFlyer")

# 去除重複
selected_cols = list(dict.fromkeys(selected_cols))

print("✅ build_label_encoding_features 需要的欄位：", selected_cols)

test_filled = test_filled.select(selected_cols)
with open("data/extra_features/transform_config_rank.pkl", "rb") as f:
    config = pickle.load(f)

# 執行編碼
df_encoded, config = build_label_encoding_features(
    df=test_filled,
    output_dir="data/extra_features/test/",
    transform_config = config
)


✅ build_label_encoding_features 需要的欄位： ['Id', 'legs0_segments0_aircraft_code', 'legs0_segments1_aircraft_code', 'legs0_segments2_aircraft_code', 'legs0_segments3_aircraft_code', 'legs1_segments0_aircraft_code', 'legs1_segments1_aircraft_code', 'legs1_segments2_aircraft_code', 'legs1_segments3_aircraft_code', 'legs0_segments0_flightNumber', 'legs0_segments1_flightNumber', 'legs0_segments2_flightNumber', 'legs0_segments3_flightNumber', 'legs1_segments0_flightNumber', 'legs1_segments1_flightNumber', 'legs1_segments2_flightNumber', 'legs1_segments3_flightNumber', 'legs0_segments0_arrivalTo_airport_city_iata', 'legs0_segments0_arrivalTo_airport_iata', 'legs0_segments0_departureFrom_airport_iata', 'legs0_segments1_arrivalTo_airport_city_iata', 'legs0_segments1_arrivalTo_airport_iata', 'legs0_segments1_departureFrom_airport_iata', 'legs0_segments2_arrivalTo_airport_city_iata', 'legs0_segments2_arrivalTo_airport_iata', 'legs0_segments2_departureFrom_airport_iata', 'legs0_segments3_arrivalTo_ai

# add view time

In [3]:
from scripts.feature_enigeer  import enrich_flight_view_features
# 所需欄位 (補齊所有必要欄位)
needed_cols = [
    "Id",
    "ranker_id",
    # segment columns for legs0
    "legs0_segments0_departureFrom_airport_iata",
    "legs0_segments0_arrivalTo_airport_iata",
    "legs0_segments1_departureFrom_airport_iata",
    "legs0_segments1_arrivalTo_airport_iata",
    "legs0_segments2_departureFrom_airport_iata",
    "legs0_segments2_arrivalTo_airport_iata",
    "legs0_segments3_departureFrom_airport_iata",
    "legs0_segments3_arrivalTo_airport_iata",
    # segment columns for legs1
    "legs1_segments0_departureFrom_airport_iata",
    "legs1_segments0_arrivalTo_airport_iata",
    "legs1_segments1_departureFrom_airport_iata",
    "legs1_segments1_arrivalTo_airport_iata",
    "legs1_segments2_departureFrom_airport_iata",
    "legs1_segments2_arrivalTo_airport_iata",
    "legs1_segments3_departureFrom_airport_iata",
    "legs1_segments3_arrivalTo_airport_iata"
]



# 篩選
train_filled = train_filled.select([c for c in needed_cols if c in train_filled.columns])

# 呼叫 enrich_flight_view_features
train_filled, _ = enrich_flight_view_features(
    train_filled,
    output_dir="data/extra_features/train/",

)

✅ 已儲存flight view特徵: data/extra_features/train/11_flight_view_features.parquet
✅ 已儲存 transform_config: data/extra_features/train/transform_flight_view_key_config.pkl


In [4]:
train_filled

Id,ranker_id,legs0_segments0_key_view_count,legs0_segments1_key_view_count,legs0_segments2_key_view_count,legs0_segments3_key_view_count,legs1_segments0_key_view_count,legs1_segments1_key_view_count,legs1_segments2_key_view_count,legs1_segments3_key_view_count,leg0_flight_view_count,leg1_flight_view_count,all_flight_view_count,leg0_view_norm,leg1_view_norm,all_view_norm,leg0_view_diff_mean,leg1_view_diff_mean,all_view_diff_mean,leg0_flight_view_count_rank,leg1_flight_view_count_rank,all_flight_view_count_rank,legs0_segments0_key_view_count_rank,legs0_segments1_key_view_count_rank,legs0_segments2_key_view_count_rank,legs0_segments3_key_view_count_rank,legs1_segments0_key_view_count_rank,legs1_segments1_key_view_count_rank,legs1_segments2_key_view_count_rank,legs1_segments3_key_view_count_rank
i64,str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,f64,f64,f64,f64,f64,f64,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,"""98ce0dabf6964640b63079fbafd42c…",64,0,0,0,87,0,0,0,19,7,1,0.204301,0.291667,0.041667,-71.04,-16.32,-22.08,1,1,1,1,1,1,1,1,1,1,1
1,"""98ce0dabf6964640b63079fbafd42c…",1270,99061,0,0,106835,1171,0,0,93,24,24,1.0,1.0,1.0,2.96,0.68,0.92,2,2,2,2,2,1,1,2,2,1,1
2,"""98ce0dabf6964640b63079fbafd42c…",1270,99061,0,0,106835,1171,0,0,93,24,24,1.0,1.0,1.0,2.96,0.68,0.92,2,2,2,2,2,1,1,2,2,1,1
3,"""98ce0dabf6964640b63079fbafd42c…",1270,99061,0,0,106835,1171,0,0,93,24,24,1.0,1.0,1.0,2.96,0.68,0.92,2,2,2,2,2,1,1,2,2,1,1
4,"""98ce0dabf6964640b63079fbafd42c…",1270,99061,0,0,106835,1171,0,0,93,24,24,1.0,1.0,1.0,2.96,0.68,0.92,2,2,2,2,2,1,1,2,2,1,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
18146427,"""88f8c53a28bf4f438941fd67338009…",43591,9089,0,0,10699,33616,0,0,1542,293,83,0.00467,0.000982,0.000297,-215980.654237,-186331.245763,-176213.166102,2,1,3,4,2,1,1,2,2,1,1
18146428,"""88f8c53a28bf4f438941fd67338009…",36102,13035,0,0,8864,41668,0,0,1654,701,232,0.005009,0.002348,0.000829,-215868.654237,-185923.245763,-176064.166102,3,2,6,2,3,1,1,1,3,1,1
18146429,"""88f8c53a28bf4f438941fd67338009…",36102,13035,0,0,10699,33616,0,0,1654,293,90,0.005009,0.000982,0.000322,-215868.654237,-186331.245763,-176206.166102,3,1,4,2,3,1,1,2,2,1,1


In [3]:
from scripts.feature_enigeer  import enrich_flight_view_features
import pickle
# 所需欄位 (補齊所有必要欄位)
needed_cols = [
    "Id",
    "ranker_id",
    # segment columns for legs0
    "legs0_segments0_departureFrom_airport_iata",
    "legs0_segments0_arrivalTo_airport_iata",
    "legs0_segments1_departureFrom_airport_iata",
    "legs0_segments1_arrivalTo_airport_iata",
    "legs0_segments2_departureFrom_airport_iata",
    "legs0_segments2_arrivalTo_airport_iata",
    "legs0_segments3_departureFrom_airport_iata",
    "legs0_segments3_arrivalTo_airport_iata",
    # segment columns for legs1
    "legs1_segments0_departureFrom_airport_iata",
    "legs1_segments0_arrivalTo_airport_iata",
    "legs1_segments1_departureFrom_airport_iata",
    "legs1_segments1_arrivalTo_airport_iata",
    "legs1_segments2_departureFrom_airport_iata",
    "legs1_segments2_arrivalTo_airport_iata",
    "legs1_segments3_departureFrom_airport_iata",
    "legs1_segments3_arrivalTo_airport_iata"
]



# 篩選
test_filled = test_filled.select([c for c in needed_cols if c in test_filled.columns])

with open("data/extra_features/transform_flight_view_key_config.pkl", "rb") as f:
    config = pickle.load(f)
    
# 呼叫 enrich_flight_view_features
test_filled, _ = enrich_flight_view_features(
    test_filled,
    output_dir="data/extra_features/test/",
    transform_config=config
)

✅ 已儲存flight view特徵: data/extra_features/test/11_flight_view_features.parquet


# Add company ID feature

In [ ]:
from scripts.feature_enigeer  import build_company_loo_features

cols_to_keep = [
    "Id",
    "companyID",
    "ranker_id",
    "selected",
    "totalPrice",
    "taxes",
    "legs0_duration",
    "legs1_duration",
    "legs0_departureAt",
    "legs0_arrivalAt",
    "legs1_departureAt",
    "legs1_arrivalAt",
    "legs0_segments0_cabinClass",
    "legs0_segments1_departureFrom_airport_iata",
    "legs0_segments2_departureFrom_airport_iata",
    "legs0_segments3_departureFrom_airport_iata",
    "legs1_segments1_departureFrom_airport_iata",
    "legs1_segments2_departureFrom_airport_iata",
    "legs1_segments3_departureFrom_airport_iata"
]

train_filled = train_filled.select(cols_to_keep)

train_new, transform_dict = build_company_loo_features(train_filled,
                                                       output_dir= "data/extra_features/train")


In [ ]:
from scripts.feature_enigeer  import build_company_loo_features
cols_to_keep = [
    "Id",
    "companyID",
    "ranker_id",
    "selected",
    "totalPrice",
    "taxes",
    "legs0_duration",
    "legs1_duration",
    "legs0_departureAt",
    "legs0_arrivalAt",
    "legs1_departureAt",
    "legs1_arrivalAt",
    "legs0_segments0_cabinClass",
    "legs0_segments1_departureFrom_airport_iata",
    "legs0_segments2_departureFrom_airport_iata",
    "legs0_segments3_departureFrom_airport_iata",
    "legs1_segments1_departureFrom_airport_iata",
    "legs1_segments2_departureFrom_airport_iata",
    "legs1_segments3_departureFrom_airport_iata"
]

test_filled = test_filled.select(cols_to_keep)
test_new, transform_dict = build_company_loo_features(test_filled,
                                                       output_dir= "data/extra_features/test",
                                                       transform_dict=transform_dict)


# Add cluster features

In [ ]:
from scripts.feature_enigeer import build_cluster_transform_dict
transform_dict = build_cluster_transform_dict(
    transform_path="data/extra_features/transform_dict_companyID.pkl",
    output_path="data/extra_features/transform_dict_cluster.pkl",
    k=3
)

In [ ]:
from scripts.feature_enigeer import add_cluster_features_and_save


needed_cols = ["Id","companyID"]
# 篩選
train_filled = train_filled.select([c for c in needed_cols if c in train_filled.columns])


df_with_cluster = add_cluster_features_and_save(
    df=train_filled,
    transform_dict_path="data/extra_features/transform_dict_cluster.pkl",
    output_dir="data/extra_features/train"
)


In [3]:
from scripts.feature_enigeer import add_cluster_features_and_save


needed_cols = ["Id","companyID"]
# 篩選
test_filled = test_filled.select([c for c in needed_cols if c in test_filled.columns])


df_with_cluster = add_cluster_features_and_save(
    df=test_filled,
    transform_dict_path="data/extra_features/transform_dict_cluster.pkl",
    output_dir="data/extra_features/test"
)


✅ 已儲存 cluster features: data/extra_features/test/13_cluster_features.parquet
shape: (5, 18)
┌──────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ Id       ┆ companyID ┆ cluster_l ┆ totalPric ┆ … ┆ mode_cabi ┆ mode_has_ ┆ mode_tran ┆ total_occ │
│ ---      ┆ ---       ┆ abel      ┆ e_mean_me ┆   ┆ n_class_m ┆ transfer_ ┆ sfer_num_ ┆ urrences_ │
│ i64      ┆ i64       ┆ ---       ┆ an        ┆   ┆ ean       ┆ mean      ┆ mean      ┆ mean      │
│          ┆           ┆ i64       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│          ┆           ┆           ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
╞══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 18144679 ┆ 62840     ┆ 0         ┆ 31749.003 ┆ … ┆ 1.036313  ┆ 0.0       ┆ 0.0       ┆ 25450.321 │
│          ┆           ┆           ┆ 054       ┆   ┆           ┆           ┆           ┆ 229       │

# remove time

In [1]:
from scripts.feature_enigeer import merge_original_with_extra_features, clean_fill_and_cast_columns

merged_df = merge_original_with_extra_features(
    base_parquet_path="data/train.parquet",
    extra_features_dir="data/extra_features/train/"
)

# 如果要儲存
merged_df  = clean_fill_and_cast_columns(merged_df)
merged_df.write_parquet("data/train_filled_v1_with_cluster.parquet")

✅ 讀取原始資料: data/train.parquet
✅ 共找到 13 個 Parquet 要合併
🔹 合併第 1/13 個: data/extra_features/train/1_price_features.parquet
🔹 合併第 2/13 個: data/extra_features/train/2_duration_features.parquet
⚠️ 10 個特徵將被新檔案覆蓋: ['legs0_segments3_duration', 'legs0_segments2_duration', 'legs1_segments1_duration', 'legs1_segments3_duration', 'legs0_segments0_duration', 'legs0_segments1_duration', 'legs1_segments2_duration', 'legs1_segments0_duration', 'legs0_duration', 'legs1_duration']
🔹 合併第 3/13 個: data/extra_features/train/3_frequent_flyer_features.parquet
⚠️ 40 個特徵將被新檔案覆蓋: ['legs0_segments3_duration_rank', 'legs0_segments3_duration', 'legs0_segments1_operatingCarrier_code', 'legs1_segments0_operatingCarrier_code', 'legs0_segments2_marketingCarrier_code', 'legs0_segments1_marketingCarrier_code', 'legs1_segments3_operatingCarrier_code', 'legs0_duration_rank', 'ranker_id', 'legs1_segments1_operatingCarrier_code', 'legs1_segments1_marketingCarrier_code', 'legs1_segments2_duration_rank', 'legs0_segments0_marketing

In [6]:
from scripts.feature_enigeer import merge_original_with_extra_features,clean_fill_and_cast_columns
merged_df = merge_original_with_extra_features(
    base_parquet_path="data/test.parquet",
    extra_features_dir="data/extra_features/test/")

# 如果要儲存
merged_df  = clean_fill_and_cast_columns(merged_df)

merged_df.write_parquet("data/test_filled_v1_with_cluster.parquet")

✅ 讀取原始資料: data/test.parquet
✅ 共找到 13 個 Parquet 要合併
🔹 合併第 1/13 個: data/extra_features/test/1_price_features.parquet
🔹 合併第 2/13 個: data/extra_features/test/2_duration_features.parquet
⚠️ 10 個特徵將被新檔案覆蓋: ['legs0_segments3_duration', 'legs1_segments3_duration', 'legs0_duration', 'legs1_duration', 'legs0_segments2_duration', 'legs1_segments0_duration', 'legs1_segments2_duration', 'legs0_segments0_duration', 'legs1_segments1_duration', 'legs0_segments1_duration']
🔹 合併第 3/13 個: data/extra_features/test/3_frequent_flyer_features.parquet
⚠️ 40 個特徵將被新檔案覆蓋: ['legs1_segments2_marketingCarrier_code', 'legs0_duration', 'legs1_segments1_operatingCarrier_code', 'legs1_segments3_marketingCarrier_code', 'legs1_segments1_duration_rank', 'legs0_duration_rank', 'legs1_segments1_duration', 'legs0_segments3_marketingCarrier_code', 'legs0_segments2_marketingCarrier_code', 'legs1_segments3_duration', 'legs0_segments2_duration', 'legs0_segments2_operatingCarrier_code', 'legs1_segments0_duration_rank', 'legs1_seg

In [2]:
# 找字串欄
import polars as pl
str_cols = [c for c in merged_df.columns if merged_df[c].dtype in (pl.Utf8, pl.String)]
# 找數值欄
numeric_cols = [c for c in merged_df.columns if merged_df[c].dtype.is_numeric()]
# 找布林欄
bool_cols = [c for c in merged_df.columns if merged_df[c].dtype == pl.Boolean]

print(f"✅ 共找到 {len(str_cols)} 個字串欄位")
print(f"✅ 共找到 {len(numeric_cols)} 個數值欄位")
print(f"✅ 共找到 {len(bool_cols)} 個布林欄位")


✅ 共找到 6 個字串欄位
✅ 共找到 271 個數值欄位
✅ 共找到 0 個布林欄位


In [4]:
merged_df[str_cols]

frequentFlyer,legs0_departureAt,legs0_arrivalAt,legs1_departureAt,legs1_arrivalAt,ranker_id
str,str,str,str,str,str
"""missing""","""2024-12-19T06:50:00""","""2024-12-19T11:20:00""","""2024-12-21T21:10:00""","""2024-12-21T21:55:00""","""c9373e5f772e43d593dd6ad2fa90f6…"
"""missing""","""2024-12-19T06:50:00""","""2024-12-19T11:20:00""","""2024-12-21T21:10:00""","""2024-12-21T21:55:00""","""c9373e5f772e43d593dd6ad2fa90f6…"
"""missing""","""2024-12-19T06:50:00""","""2024-12-19T11:20:00""","""2024-12-21T21:10:00""","""2024-12-21T21:55:00""","""c9373e5f772e43d593dd6ad2fa90f6…"
"""missing""","""2024-12-19T08:25:00""","""2024-12-19T12:45:00""","""2024-12-21T12:00:00""","""2024-12-21T12:35:00""","""c9373e5f772e43d593dd6ad2fa90f6…"
"""missing""","""2024-12-19T08:25:00""","""2024-12-19T12:45:00""","""2024-12-21T12:00:00""","""2024-12-21T12:35:00""","""c9373e5f772e43d593dd6ad2fa90f6…"
…,…,…,…,…,…
"""missing""","""2025-01-08T09:05:00""","""2025-01-08T12:50:00""","""missing""","""missing""","""c5622e0de0594bde95a4dd8c1fcff7…"
"""missing""","""2025-01-08T21:25:00""","""2025-01-09T01:10:00""","""missing""","""missing""","""c5622e0de0594bde95a4dd8c1fcff7…"
"""missing""","""2025-01-08T21:25:00""","""2025-01-09T01:10:00""","""missing""","""missing""","""c5622e0de0594bde95a4dd8c1fcff7…"


In [1]:
import polars as pl

# 讀 parquet
test = pl.read_parquet('data/test_filled_v1_with_cluster.parquet')
train = pl.read_parquet('data/train_filled_v1_with_cluster.parquet')

# 比較欄位
train_cols = set(train.columns)
test_cols = set(test.columns)

print("✅ Train columns:", len(train_cols))
print("✅ Test columns:", len(test_cols))

print("\n🎯 Train 中有但 Test 沒有的欄位:")
print(sorted(train_cols - test_cols))

print("\n🎯 Test 中有但 Train 沒有的欄位:")
print(sorted(test_cols - train_cols))

if train_cols == test_cols:
    print("\n✅ Train 和 Test 欄位完全一致！")

# 比較 companyID
train_company_ids = set(train["companyID"].unique().to_list())
test_company_ids = set(test["companyID"].unique().to_list())

only_in_test = sorted(test_company_ids - train_company_ids)

print("\n🎯 Test 中有但 Train 沒有的 companyID:")
print(only_in_test)

print(f"\n✅ Test unique companyID: {len(test_company_ids)}")
print(f"✅ Train unique companyID: {len(train_company_ids)}")
print(f"✅ Test 但不在 Train 的 companyID 數量: {len(only_in_test)}")


✅ Train columns: 295
✅ Test columns: 294

🎯 Train 中有但 Test 沒有的欄位:
['selected']

🎯 Test 中有但 Train 沒有的欄位:
[]

🎯 Test 中有但 Train 沒有的 companyID:
[16880, 37962, 43019, 45683, 46743, 52514, 54908, 57310, 59070, 59090, 59277, 59487, 59818, 60219, 60288, 60602, 60838, 61579, 61607, 61627, 61641, 61654, 61709, 61724, 61751, 61810, 61818, 61822, 61832, 61874, 62020, 62029, 62098, 62110, 62368, 62459, 62463, 62794, 62871, 63196, 63243]

✅ Test unique companyID: 495
✅ Train unique companyID: 641
✅ Test 但不在 Train 的 companyID 數量: 41
